In [ ]:
import numpy as np
import pandas as pd

import sklearn
import sklearn.metrics
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, auc, precision_score, recall_score, f1_score, classification_report, accuracy_score, confusion_matrix, ConfusionMatrixDisplay, matthews_corrcoef
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve
from sklearn.feature_selection import RFE

from scipy import stats as st
from random import randrange
from matplotlib import pyplot

import xgboost as xgb
import lightgbm as lgb
import catboost as cb
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

import optuna

import shap

In [ ]:
#Open csv file.

data = pd.read_csv("./DMVO-NIHSS Shift/dmvo_final.csv", index_col = 0)
data.head()

In [ ]:
columns = data.columns.tolist()

print(columns)

In [ ]:
#Define predictor variables and outcome of interest.

variables = ['AGE', 'GLUCOSE', 'HR', 'SBP', 'DBP', 'SODIUM', 'POTASSIUM', 'CALCIUM', 'BUNCR', 'HGB', 'HTC', 'PLT', 'NEUPLT', 'MPV', 'MCV', 'ASPECTS', 'TIME_GROIN', 'RCBF20', 'RCBF30', 'RCBF34', 'RCBF38', 'TMAX4', 'TMAX6', 'TMAX8', 'TMAX10', 'CBV34', 'CBV38', 'CBV42', 'MISMATCH_VOL', 'HYPOPERF_INDEX', 'DWI_VOL', 'INFARCT_GROWTH', 'POST_NCCT', 'SEX_Female', 'SEX_Male', 'RACE_Black/African American', 'RACE_Other', 'RACE_White', 'AFIB_No', 'AFIB_Yes', 'DIABETES_No', 'DIABETES_Yes', 'HYPERTENSION_No', 'HYPERTENSION_Yes', 'MALIGNANCY_No', 'MALIGNANCY_Yes', 'CAD_No', 'CAD_Yes', 'IVTPA_No', 'IVTPA_Yes', 'HEMTRANS_No', 'HEMTRANS_Yes', 'THROMBECTOMY_No', 'THROMBECTOMY_Yes', 'PASS_1', 'PASS_2', 'PASS_3', 'PASS_4', 'PASS_Not attempted', 'MTICI_1', 'MTICI_2b', 'MTICI_2c', 'MTICI_3', 'MTICI_Not attempted', 'VESSEL_ACA', 'VESSEL_MCA', 'VESSEL_PCA', 'VESSEL_PICA', 'LATERALITY_Left', 'LATERALITY_Right', 'SEGMENT_A2', 'SEGMENT_M1-M2', 'SEGMENT_M2', 'SEGMENT_M3', 'SEGMENT_P1', 'SEGMENT_P2', 'SEGMENT_P3', 'LOCATION_Distal', 'LOCATION_Medial', 'LOCATION_Proximal', 'LOCATION_Unknown', 'NIHSS_SHIFT_CAT_Low shift']

In [ ]:
#Redefine data.

data = data[variables]

In [ ]:
#Define predictor variables (x) and outcome of interest (y).

x = data.drop(['NIHSS_SHIFT_CAT_Low shift'], axis = 1)
y = data['NIHSS_SHIFT_CAT_Low shift']

In [ ]:
#Check data shapes.

print(y.shape)
print(x.shape)

In [ ]:
#Split data into train and test sets in 70:30 ratio.

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 42)

#Describe train and test sets.

print("Number patients x_train dataset: ", x_train.shape)
print("Number patients y_train dataset: ", y_train.shape)
print("Number patients x_test dataset: ", x_test.shape)
print("Number patients y_test dataset: ", y_test.shape)

In [ ]:
#Describe outcome of interest before resampling.

print("Before resampling, counts of label '1': {}".format(sum(y_train == 1)))
print("Before resampling, counts of label '0': {} \n".format(sum(y_train == 0)))

In [ ]:
#No resampling.

x_rs = x_train
y_rs = y_train

In [ ]:
#Describe outcome of interest after resampling.

print("After resampling, counts of label '1': {}".format(sum(y_rs == 1)))
print("After resampling, counts of label '0': {} \n".format(sum(y_rs == 0)))

# Feature Selection

In [ ]:
rfe_method = RFE(
    RandomForestClassifier(n_estimators=10, random_state=10),
    n_features_to_select=10,
    step=1,
)

rfe_method.fit(x_rs, y_rs)

selected_features = list(x_rs.columns[(rfe_method.get_support())])

In [ ]:
selected_features

In [ ]:
x_rs = x_rs[selected_features]
x_test = x_test[selected_features]

# XGBoost

In [ ]:
#Hyperparameter tuning for XGBoost.

def objective(trial):
    data, target = x_rs, y_rs
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)
    dtrain = xgb.DMatrix(train_x, label=train_y)
    dvalid = xgb.DMatrix(valid_x, label=valid_y)

    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
    }

    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    # Add a callback for pruning.
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-auc")

    bst = xgb.train(param, dtrain, evals=[(dvalid, "validation")], callbacks=[pruning_callback])
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
    auc = sklearn.metrics.roc_auc_score(valid_y, pred_labels)

    return auc


if __name__ == "__main__":
    study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=5), direction="maximize"
    )
    study.optimize(objective, n_trials=100)
    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    xgb_params = {}

    for key, value in trial.params.items():
        xgb_params[key] = value

In [ ]:
#Fit XGBoost.

from xgboost import XGBClassifier

xgb = XGBClassifier(**xgb_params)

xgb.fit(x_rs,y_rs)

In [ ]:
#Make predictions on the test set based on the trained XGBoost model.

preds_xgb = xgb.predict(x_test)

probs_xgb = xgb.predict_proba(x_test)
probs_xgb = probs_xgb[:, 1]

In [ ]:
#Evaluate XGBoost model.

xgb_precision = precision_score(preds_xgb,y_test)
xgb_recall = recall_score(preds_xgb,y_test)
xgb_f1 = f1_score(preds_xgb,y_test)
xgb_acc = accuracy_score(preds_xgb,y_test)
xgb_mcc = matthews_corrcoef(y_test, preds_xgb)
xgb_auroc = roc_auc_score(y_test, probs_xgb)

print("Precision: %.3f" % (xgb_precision))
print("Recall: %.3f" % (xgb_recall))
print("F1 Score: %.3f" %(xgb_f1))
print('Accuracy: %.3f' % (xgb_acc))
print('MCC: %.3f' % (xgb_mcc))
print('AUROC: %.3f' % (xgb_auroc))

In [ ]:
#Evaluate XGBoost model (PRC and AUPRC).

xgb_precision, xgb_recall, _ = precision_recall_curve(y_test, probs_xgb)
xgb_auprc = auc(xgb_recall, xgb_precision)

print('AUPRC: %.3f' % (xgb_auprc))

In [ ]:
#Recalculate precision and recall for calculation purposes.

xgb_precision = precision_score(preds_xgb,y_test)
xgb_recall = recall_score(preds_xgb,y_test)

xgb_results = [xgb_precision, xgb_recall, xgb_f1, xgb_acc, xgb_mcc, xgb_auroc, xgb_auprc]

In [ ]:
#Recalculate precision recall curve for plotting purposes.

xgb_precision, xgb_recall, _ = precision_recall_curve(y_test, probs_xgb)

# LightGBM

In [ ]:
#Hyperparameter tuning for LightGBM.

def objective(trial):
    data, target = x_rs, y_rs
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)
    dtrain = lgb.Dataset(train_x, label=train_y)

    param = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    gbm = lgb.train(param, dtrain)
    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)
    auc = sklearn.metrics.roc_auc_score(valid_y, pred_labels)
    return auc


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    lgb_params = {}

    for key, value in trial.params.items():
        lgb_params[key] = value

In [ ]:
#Fit LightGBM.

import lightgbm as lgb

lgb = lgb.LGBMClassifier(**lgb_params)

lgb.fit(x_rs, y_rs)

In [ ]:
#Make predictions on the test set based on the trained model.

preds_lgb = lgb.predict(x_test)

probs_lgb = lgb.predict_proba(x_test)
probs_lgb = probs_lgb[:, 1]

In [ ]:
#Evaluate LightGBM model.

lgb_precision = precision_score(preds_lgb,y_test)
lgb_recall = recall_score(preds_lgb,y_test)
lgb_f1 = f1_score(preds_lgb,y_test)
lgb_acc = accuracy_score(preds_lgb,y_test)
lgb_mcc = matthews_corrcoef(y_test, preds_lgb)
lgb_auroc = roc_auc_score(y_test, probs_lgb)

print("Precision: %.3f" % (lgb_precision))
print("Recall: %.3f" % (lgb_recall))
print("F1 Score: %.3f" %(lgb_f1))
print('Accuracy: %.3f' % (lgb_acc))
print('MCC: %.3f' % (lgb_mcc))
print('AUROC: %.3f' % (lgb_auroc))

In [ ]:
#Evaluate LightGBM model (PRC and AUPRC).

lgb_precision, lgb_recall, _ = precision_recall_curve(y_test, probs_lgb)
lgb_auprc = auc(lgb_recall, lgb_precision)

print('AUPRC: %.3f' % (lgb_auprc))

In [ ]:
#Recalculate precision and recall for calculation purposes.

lgb_precision = precision_score(preds_lgb,y_test)
lgb_recall = recall_score(preds_lgb,y_test)

lgb_results = [lgb_precision, lgb_recall, lgb_f1, lgb_acc, lgb_mcc, lgb_auroc, lgb_auprc]

In [ ]:
#Recalculate precision recall curve for plotting purposes.

lgb_precision, lgb_recall, _ = precision_recall_curve(y_test, probs_lgb)

# CatBoost

In [ ]:
#Hyperparameter tuning for CatBoost.

from optuna.integration import CatBoostPruningCallback

def objective(trial: optuna.Trial) -> float:
    data, target = x_rs, y_rs
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)

    param = {
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "used_ram_limit": "3gb",
        "eval_metric": "AUC",
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1, log=True)

    gbm = cb.CatBoostClassifier(**param)

    pruning_callback = CatBoostPruningCallback(trial, "AUC")
    gbm.fit(
        train_x,
        train_y,
        eval_set=[(valid_x, valid_y)],
        verbose=0,
        early_stopping_rounds=100,
        callbacks=[pruning_callback],
    )

    # evoke pruning manually.
    pruning_callback.check_pruned()

    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)
    auc = sklearn.metrics.roc_auc_score(valid_y, pred_labels)

    return auc

if __name__ == "__main__":
    study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=5), direction="maximize"
    )
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    cb_params = {}

    for key, value in trial.params.items():
        cb_params[key] = value

In [ ]:
#Fit CatBoost.

from catboost import CatBoostClassifier

cb = CatBoostClassifier(**cb_params)

cb.fit(x_rs,y_rs)

In [ ]:
#Make predictions on the test set based on the trained model.

preds_cb = cb.predict(x_test)

probs_cb = cb.predict_proba(x_test)
probs_cb = probs_cb[:, 1]

In [ ]:
#Evaluate CatBoost model.

cb_precision = precision_score(preds_cb,y_test)
cb_recall = recall_score(preds_cb,y_test)
cb_f1 = f1_score(preds_cb,y_test)
cb_acc = accuracy_score(preds_cb,y_test)
cb_mcc = matthews_corrcoef(y_test, preds_cb)
cb_auroc = roc_auc_score(y_test, probs_cb)

print("Precision: %.3f" % (cb_precision))
print("Recall: %.3f" % (cb_recall))
print("F1 Score: %.3f" %(cb_f1))
print('Accuracy: %.3f' % (cb_acc))
print('MCC: %.3f' % (cb_mcc))
print('AUROC: %.3f' % (cb_auroc))

In [ ]:
#Evaluate XGBoost model (PRC and AUPRC).

cb_precision, cb_recall, _ = precision_recall_curve(y_test, probs_cb)
cb_auprc = auc(cb_recall, cb_precision)

print('AUPRC: %.3f' % (cb_auprc))

In [ ]:
#Recalculate precision and recall for calculation purposes.

cb_precision = precision_score(preds_cb,y_test)
cb_recall = recall_score(preds_cb,y_test)

cb_results = [cb_precision, cb_recall, cb_f1, cb_acc, cb_mcc, cb_auroc, cb_auprc]

In [ ]:
#Recalculate precision recall curve for plotting purposes.

cb_precision, cb_recall, _ = precision_recall_curve(y_test, probs_cb)

# Multi-Layer Perceptron

In [ ]:
#Hyperparameter tuning for Multi-Layer Perceptron.

def objective(trial: optuna.Trial) -> float:
    data, target = x_rs, y_rs
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)

    param = {
        "alpha": trial.suggest_float("alpha", 1, 12),
        "activation": trial.suggest_categorical("activation", ["tanh", "relu"]),
        "solver": trial.suggest_categorical("solver", ["sgd", "adam"]),
        "learning_rate": trial.suggest_categorical("learning_rate", ["constant", "adaptive","invscaling"]),
    }

    mlp = MLPClassifier(**param)

    mlp.fit(
        train_x,
        train_y,
    )

    preds = mlp.predict(valid_x)
    pred_labels = np.rint(preds)
    auc = sklearn.metrics.roc_auc_score(valid_y, pred_labels)

    return auc


if __name__ == "__main__":
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

In [ ]:
#Fit MLP.

from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(alpha=1.0026228802456412, activation = 'tanh', solver = 'adam', learning_rate = 'adaptive')

mlp.fit(x_rs, y_rs)

In [ ]:
#Make predictions on the test set based on the trained model.

preds_mlp = mlp.predict(x_test)

probs_mlp = mlp.predict_proba(x_test)
probs_mlp = probs_mlp[:, 1]

In [ ]:
#Evaluate MLP model.

mlp_precision = precision_score(preds_mlp,y_test)
mlp_recall = recall_score(preds_mlp,y_test)
mlp_f1 = f1_score(preds_mlp,y_test)
mlp_acc = accuracy_score(preds_mlp,y_test)
mlp_mcc = matthews_corrcoef(y_test, preds_mlp)
mlp_auroc = roc_auc_score(y_test, probs_mlp)

print("Precision: %.3f" % (mlp_precision))
print("Recall: %.3f" % (mlp_recall))
print("F1 Score: %.3f" %(mlp_f1))
print('Accuracy: %.3f' % (mlp_acc))
print('MCC: %.3f' % (mlp_mcc))
print('AUROC: %.3f' % (mlp_auroc))

In [ ]:
#Evaluate MLP model (PRC and AUPRC).

mlp_precision, mlp_recall, _ = precision_recall_curve(y_test, probs_mlp)
mlp_auprc = auc(mlp_recall, mlp_precision)

print('AUPRC: %.3f' % (mlp_auprc))

In [ ]:
#Recalculate precision and recall for calculation purposes.

mlp_precision = precision_score(preds_mlp,y_test)
mlp_recall = recall_score(preds_mlp,y_test)

mlp_results = [mlp_precision, mlp_recall, mlp_f1, mlp_acc, mlp_mcc, mlp_auroc, mlp_auprc]

In [ ]:
#Recalculate precision recall curve for plotting purposes.

mlp_precision, mlp_recall, _ = precision_recall_curve(y_test, probs_mlp)

# Random Forest

In [ ]:
#Hyperparameter tuning for Random Forest.

def objective(trial):
    data, target = x_rs, y_rs
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)

    param = {
        "criterion": trial.suggest_categorical("criterion", ["gini", "entropy"]),
        "max_features": trial.suggest_categorical("max_features", ["auto", "sqrt","log2", None]),
        "max_depth": trial.suggest_int("max_depth", 1, 100),
        "n_estimators": trial.suggest_int("n_estimators", 100, 2000, 100),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 4, 1),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 10, 1),
    }

    rf = RandomForestClassifier(**param)

    rf.fit(
        train_x,
        train_y,
    )

    preds = rf.predict(valid_x)
    pred_labels = np.rint(preds)
    auc = sklearn.metrics.roc_auc_score(valid_y, pred_labels)

    return auc


if __name__ == "__main__":
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    rf_params = {}

    for key, value in trial.params.items():
        rf_params[key] = value

In [ ]:
#Fit Random Forest.

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(**rf_params)

rf.fit(x_rs,y_rs)

In [ ]:
#Make predictions on the test set based on the trained model.

preds_rf = rf.predict(x_test)

probs_rf = rf.predict_proba(x_test)
probs_rf = probs_rf[:, 1]

In [ ]:
#Evaluate Random Forest model.

rf_precision = precision_score(preds_rf,y_test)
rf_recall = recall_score(preds_rf,y_test)
rf_f1 = f1_score(preds_rf,y_test)
rf_acc = accuracy_score(preds_rf,y_test)
rf_mcc = matthews_corrcoef(y_test, preds_rf)
rf_auroc = roc_auc_score(y_test, probs_rf)

print("Precision: %.3f" % (rf_precision))
print("Recall: %.3f" % (rf_recall))
print("F1 Score: %.3f" %(rf_f1))
print('Accuracy: %.3f' % (rf_acc))
print('MCC: %.3f' % (rf_mcc))
print('AUROC: %.3f' % (rf_auroc))

In [ ]:
#Evaluate Random Forest model (PRC and AUPRC).

rf_precision, rf_recall, _ = precision_recall_curve(y_test, probs_rf)
rf_auprc = auc(rf_recall, rf_precision)

print('AUPRC: %.3f' % (rf_auprc))

In [ ]:
#Recalculate precision and recall for calculation purposes.

rf_precision = precision_score(preds_rf,y_test)
rf_recall = recall_score(preds_rf,y_test)

rf_results = [rf_precision, rf_recall, rf_f1, rf_acc, rf_mcc, rf_auroc, rf_auprc]

In [ ]:
#Recalculate precision recall curve for plotting purposes.

rf_precision, rf_recall, _ = precision_recall_curve(y_test, probs_rf)

#Logistic Regression

In [ ]:
from optuna.samplers import TPESampler


In [ ]:
# Hyperparameter tuning for Logistic Regression.

def objective(trial):
    data, target = x_rs, y_rs
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)

    param = {
        "penalty": trial.suggest_categorical("penalty", ["l2"]),
        "C": trial.suggest_float("C", 1e-4, 1e4, log=True),
        "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False]),
        "solver": trial.suggest_categorical("solver", ["newton-cg", "lbfgs", "liblinear", "sag", "saga"]),
        "max_iter": trial.suggest_int("max_iter", 100, 2000, 100),
        "random_state": 31,
    }

    logreg = LogisticRegression(**param)

    logreg.fit(
        train_x,
        train_y,
    )

    preds = logreg.predict(valid_x)
    pred_labels = np.rint(preds)
    auc = sklearn.metrics.roc_auc_score(valid_y, pred_labels)

    return auc


if __name__ == "__main__":
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    logreg_params = {}

    for key, value in trial.params.items():
        logreg_params[key] = value


In [ ]:
#Fit Logistic Regression.

from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(**logreg_params)

logreg.fit(x_rs,y_rs)

In [ ]:
#Make predictions on the test set based on the trained model.

preds_logreg = logreg.predict(x_test)

probs_logreg = logreg.predict_proba(x_test)
probs_logreg = probs_logreg[:, 1]

In [ ]:
#Evaluate Logistic Regression model.

logreg_precision = precision_score(preds_logreg,y_test)
logreg_recall = recall_score(preds_logreg,y_test)
logreg_f1 = f1_score(preds_logreg,y_test)
logreg_acc = accuracy_score(preds_logreg,y_test)
logreg_mcc = matthews_corrcoef(y_test, preds_logreg)
logreg_auroc = roc_auc_score(y_test, probs_logreg)

print("Precision: %.3f" % (logreg_precision))
print("Recall: %.3f" % (logreg_recall))
print("F1 Score: %.3f" %(logreg_f1))
print('Accuracy: %.3f' % (logreg_acc))
print('MCC: %.3f' % (logreg_mcc))
print('AUROC: %.3f' % (logreg_auroc))

In [ ]:
#Evaluate Logistic Regression model (PRC and AUPRC).

logreg_precision, logreg_recall, _ = precision_recall_curve(y_test, probs_logreg)
logreg_auprc = auc(logreg_recall, logreg_precision)

print('AUPRC: %.3f' % (logreg_auprc))

In [ ]:
#Recalculate precision and recall for calculation purposes.

logreg_precision = precision_score(preds_logreg,y_test)
logreg_recall = recall_score(preds_logreg,y_test)

logreg_results = [logreg_precision, logreg_recall, logreg_f1, logreg_acc, logreg_mcc, logreg_auroc, logreg_auprc]

# ROC, PR, and Calibration Plots

In [ ]:
f = pyplot.figure()
f.set_figwidth(12)
f.set_figheight(12)

xgb_fpr, xgb_tpr, _ = roc_curve(y_test, probs_xgb)
pyplot.plot(xgb_fpr, xgb_tpr, label='XGBoost AUROC: {:.3f}'.format(xgb_auroc), color='r')

lgb_fpr, lgb_tpr, _ = roc_curve(y_test, probs_lgb)
pyplot.plot(lgb_fpr, lgb_tpr, label='LightGBM AUROC: {:.3f}'.format(lgb_auroc), color='g')

cb_fpr, cb_tpr, _ = roc_curve(y_test, probs_cb)
pyplot.plot(cb_fpr, cb_tpr, label='CatBoost AUROC: {:.3f}'.format(cb_auroc), color = 'b')

mlp_fpr, mlp_tpr, _ = roc_curve(y_test, probs_mlp)
pyplot.plot(mlp_fpr, mlp_tpr, label='Multi-Layer Perceptron AUROC: {:.3f}'.format(mlp_auroc), color = 'c')

rf_fpr, rf_tpr, _ = roc_curve(y_test, probs_rf)
pyplot.plot(rf_fpr, rf_tpr, label='Random Forest AUROC: {:.3f}'.format(rf_auroc), color = 'm')

logreg_fpr, logreg_tpr, _ = roc_curve(y_test, probs_logreg)
pyplot.plot(logreg_fpr, logreg_tpr, label='Logistic Regression AUROC: {:.3f}'.format(logreg_auroc), color = 'y')

pyplot.plot([0, 1], [0, 1], linestyle = '--')

pyplot.title('Receiver Operating Characteristic Curve', loc='center', fontsize = 20, fontweight = 'heavy', pad = 20)
# axis labels
pyplot.xlabel('False Positive Rate', fontsize = 16, labelpad = 10)
pyplot.ylabel('True Positive Rate', fontsize = 16, labelpad = 10)
pyplot.tick_params(axis="y",direction="out")
pyplot.tick_params(axis="x",direction="out")

# show the legend
leg = pyplot.legend(loc = 'lower right', fontsize = 12)

# show the plot
pyplot.show()

In [ ]:
f = pyplot.figure()
f.set_figwidth(12)
f.set_figheight(12)

pyplot.plot(xgb_recall, xgb_precision, label='XGBoost AUPRC: {:.3f}'.format(mlp_auprc), color = 'r')
pyplot.plot(lgb_recall, lgb_precision, label='LightGBM AUPRC: {:.3f}'.format(lgb_auprc), color = 'g')
pyplot.plot(cb_recall, cb_precision, label='CatBoost AUPRC: {:.3f}'.format(cb_auprc), color = 'b')
pyplot.plot(mlp_recall, mlp_precision, label='Multi-Layer Perceptron AUPRC: {:.3f}'.format(mlp_auprc), color = 'c')
pyplot.plot(rf_recall, rf_precision, label='Random Forest AUPRC: {:.3f}'.format(rf_auprc), color = 'm')
pyplot.plot(logreg_recall, logreg_precision, label='Logistic Regression AUPRC: {:.3f}'.format(logreg_auprc), color = 'y')

pyplot.title('Precision Recall Curve', loc='center', fontsize = 20, fontweight = 'heavy', pad = 20)
#axis labels
pyplot.xlabel('Recall', fontsize = 16, labelpad = 10)
pyplot.ylabel('Precision', fontsize = 16, labelpad = 10)
# show the legend
leg = pyplot.legend(loc = 'lower right', fontsize = 12)

# show the plot
pyplot.show()

#SHAP Plots

In [ ]:
# Fits the explainer
lgb_explainer = shap.Explainer(lgb.predict, x_test)
xgb_explainer = shap.Explainer(xgb.predict, x_test)
cb_explainer = shap.Explainer(cb.predict, x_test)
mlp_explainer = shap.Explainer(mlp.predict, x_test)
rf_explainer = shap.Explainer(rf.predict, x_test)
logreg_explainer = shap.Explainer(logreg.predict, x_test)

# Calculates the SHAP values - It takes some time
lgb_shap_values = lgb_explainer(x_test)
xgb_shap_values = xgb_explainer(x_test)
cb_shap_values = cb_explainer(x_test)
mlp_shap_values = mlp_explainer(x_test)
rf_shap_values = rf_explainer(x_test)
logreg_shap_values = logreg_explainer(x_test)

In [ ]:
from scipy.special import softmax

def print_feature_importances_shap_values(shap_values, features):
    '''
    Prints the feature importances based on SHAP values in an ordered way
    shap_values -> The SHAP values calculated from a shap.Explainer object
    features -> The name of the features, on the order presented to the explainer
    '''
    # Calculates the feature importance (mean absolute shap value) for each feature
    importances = []
    for i in range(shap_values.values.shape[1]):
        importances.append(np.mean(np.abs(shap_values.values[:, i])))
    # Calculates the normalized version
    importances_norm = softmax(importances)
    # Organize the importances and columns in a dictionary
    feature_importances = {fea: imp for imp, fea in zip(importances, features)}
    feature_importances_norm = {fea: imp for imp, fea in zip(importances_norm, features)}
    # Sorts the dictionary
    feature_importances = {k: v for k, v in sorted(feature_importances.items(), key=lambda item: item[1], reverse = True)}
    feature_importances_norm= {k: v for k, v in sorted(feature_importances_norm.items(), key=lambda item: item[1], reverse = True)}
    # Prints the feature importances
    for k, v in feature_importances.items():
        print(f"{k} -> {v:.4f} (softmax = {feature_importances_norm[k]:.4f})")

In [ ]:
shap.plots.bar(xgb_shap_values)

In [ ]:
shap.plots.bar(lgb_shap_values)

In [ ]:
shap.plots.bar(cb_shap_values)

In [ ]:
shap.plots.bar(mlp_shap_values)

In [ ]:
shap.plots.bar(rf_shap_values)

In [ ]:
shap.plots.bar(logreg_shap_values)

#Results Summary

In [ ]:
results = {'XGBoost':xgb_results, 'LightGBM':lgb_results, 'CatBoost':cb_results, 'Multi-Layer Perceptron':mlp_results, 'Random Forest':rf_results, 'Logistic Regression':logreg_results}

results = pd.DataFrame(results, columns = ['XGBoost', 'LightGBM', 'CatBoost', 'Multi-Layer Perceptron', 'Random Forest', 'Logistic Regression'])

results

In [ ]:
results = pd.DataFrame({'XGBoost':xgb_results, 'LightGBM':lgb_results, 'CatBoost':cb_results, 'Multi-Layer Perceptron':mlp_results, 'Random Forest': rf_results, 'Logistic Regression': logreg_results})

results = results.T

results.columns = ['Precision', 'Recall', 'F1', 'Accuracy', 'MCC', 'AUROC', 'AUPRC']

results.to_csv('./DMVO-NIHSS Shift/algorithm_performance.csv')

results